<a href="https://colab.research.google.com/github/emmenets/neuralnetwork/blob/master/chihook190715.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 28282 (delta 0), reused 1 (delta 0), pack-reused 28277
Receiving objects: 100% (28282/28282), 509.37 MiB | 33.59 MiB/s, done.
Resolving deltas: 100% (17432/17432), done.
Checking out files: 100% (3005/3005), done.


In [2]:
!wget https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

--2019-07-15 12:18:32--  https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘detection_model_zoo.md’

detection_model_zoo     [ <=>                ] 106.30K  --.-KB/s    in 0.04s   

2019-07-15 12:18:32 (2.74 MB/s) - ‘detection_model_zoo.md’ saved [108855]



In [11]:
cd /content/models/research/object_detection

/content/models/research/object_detection


In [0]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [0]:
!tar -xf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [0]:
mkdir images

In [0]:
mkdir inference_graph

In [0]:
mkdir training

In [0]:
ls

In [10]:
cd /content/models/research/object_detection

/content/models/research/object_detection


In [0]:
!git clone https://github.com/seraj94ai/Train-Object-Detection-Classifier.git

In [0]:
!cp -R /content/models/research/object_detection/Train-Object-Detection-Classifier/errs /content/models/research/object_detection
!cp -R /content/models/research/object_detection/Train-Object-Detection-Classifier/images /content/models/research/object_detection
!cp -R /content/models/research/object_detection/Train-Object-Detection-Classifier/in /content/models/research/object_detection
!cp -R /content/models/research/object_detection/Train-Object-Detection-Classifier/out /content/models/research/object_detection
!cp  /content/models/research/object_detection/Train-Object-Detection-Classifier/Object_detection_image.py /content/models/research/object_detection
!cp  /content/models/research/object_detection/Train-Object-Detection-Classifier/Object_detection_video.py /content/models/research/object_detection
!cp  /content/models/research/object_detection/Train-Object-Detection-Classifier/generate_tfrecord.py /content/models/research/object_detection
!cp  /content/models/research/object_detection/Train-Object-Detection-Classifier/renameFiles.py /content/models/research/object_detection
!cp  /content/models/research/object_detection/Train-Object-Detection-Classifier/xml_to_csv.py /content/models/research/object_detection

In [17]:
!python3 xml_to_csv.py

Successfully converted xml to csv.
Successfully converted xml to csv.


In [20]:
%%writefile generate_tfrecord.py

"""
Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

  # Create test data:
  python generate_tfrecord.py --csv_input=images/test_labels.csv  --image_dir=images/test --output_path=test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from utils import dataset_util
from collections import namedtuple, OrderedDict

flags = tf.app.flags
flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
flags.DEFINE_string('image_dir', '', 'Path to the image directory')
flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
FLAGS = flags.FLAGS


# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'chinook':
        return 1
    else:
        return 0


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(os.getcwd(), FLAGS.image_dir)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


if __name__ == '__main__':
    tf.app.run()

Overwriting generate_tfrecord.py


In [21]:
!python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

W0715 12:33:35.144322 140312398686080 deprecation_wrapper.py:119] From generate_tfrecord.py:101: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0715 12:33:35.145220 140312398686080 deprecation_wrapper.py:119] From generate_tfrecord.py:87: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0715 12:33:35.218445 140312398686080 deprecation_wrapper.py:119] From generate_tfrecord.py:46: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/models/research/object_detection/train.record


In [22]:
!python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record

W0715 12:34:17.390953 140013441804160 deprecation_wrapper.py:119] From generate_tfrecord.py:101: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0715 12:34:17.391641 140013441804160 deprecation_wrapper.py:119] From generate_tfrecord.py:87: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0715 12:34:17.432181 140013441804160 deprecation_wrapper.py:119] From generate_tfrecord.py:46: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/models/research/object_detection/test.record


In [0]:
cp /content/models/research/object_detection/data/pet_label_map.pbtxt /content/models/research/object_detection/training

In [0]:
%%writefile training/label_map.pbtxt

item {
  id: 1
  name: 'chinook'
}

In [0]:
%%writefile training/faster_rcnn_inception_v2_pets.config

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0002
          schedule {
            step: 900000
            learning_rate: .00002
          }
          schedule {
            step: 1200000
            learning_rate: .000002
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "/content/models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt"
  from_detection_checkpoint: true
  load_all_detection_checkpoint_vars: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}


train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/models/research/object_detection/train.record"
  }
  label_map_path: "/content/models/research/object_detection/training/label_map.pbtxt"
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  num_examples: 70
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/models/research/object_detection/test.record"
  }
  label_map_path: "/content/models/research/object_detection/training/label_map.pbtxtt"
  shuffle: false
  num_readers: 1
}

Overwriting training/faster_rcnn_inception_v2_pets.config
